# Omar Ali's Submission

## Plan for Cube

> For the consideration of time, I have pre-generated RST trees from a selection of instances within the provided dataset. In practice this would be scaled up to an automated generation cycle per instance whereby, for each instance added to the database, and RST tree would be generated.
> 
> For each instance, the RST tree would be weighted with the most important parts considered highly above the less important sections. Following a similar model to text summarisation, the least important aspects will be removed.
> 
> With the most important parts identified we can train a model to determine what features are present within topics that are indicative of emerging changes required by policymakers. This model allows for our analysis to be scaled to different instances that are coming through every day. 
> 
> The hypothesis is that there is some level of consistency within information indicative of a policy change. I hope that the added level of rhetorical structure theory and its segmentation will allow for these parts to be highlighted and learned from.
> 
> In future, I may also consider using the relationships between spans of text as an indication to emerging policies. For example relationships that represent attributions or cause-and-effect might likely be an indication of emerging changes to policy.

## Implementation
1. Weighting logic to rank our RST trees
2. Data taken from the data set processes using HILDA (and RST parser)

Output:
1. Ranked, indexed list of the most important, salient points taken from the text that are likely to indicate policy changes.